# News Clustering using KMeans Algorithm
By Datetime : 2016-08-29 ~ 2016-09-05

In [7]:
import numpy as np
import pandas as pd
import articles_data_py3 as articles_data
from pymongo import MongoClient

## Load data from MongoDB 

In [5]:
client = MongoClient('mongodb://localhost:27017/somanews')
client.somanews.authenticate('ssomanews', 'ssomanews1029')
db = client.get_database('somanews')

crawled_collection = db.get_collection('crawledArticles')
clusters_collection = db.get_collection('articles')

## Select Categories
- 포함 : 경제, 문화, 건강, 과학, 사회, 정치, 스포츠
- 제외 : 종합, 정보없음, 인물, 사설

In [9]:
catelist_path = '../datastore/category.p'
train = articles_data.find_recent_articles(crawled_collection, catelist_path)

## Preprocessing
1. Datetime (16-09-11 ~ 16-09-17)
2. Remove stopwords (regex, hanja)
3. POS Tagging with KoNLPy, Mecab
4. Using bigram

In [7]:
import datetime
from konlpy.tag import Mecab
import cnouns
import hanja
import re

In [8]:
mecab = Mecab()

In [9]:
def text_cleaning(text):
    text = hanja.translate(text, 'substitution')
    text = re.sub('[^가-힝0-9a-zA-Z\\s]', ' ', text)
    return text

In [10]:
def tokenize(data):
    return [' '.join(e for e in mecab.nouns(data))]

In [11]:
train['title_flat'] = train['title'].apply(lambda text: text_cleaning(text))
train['title_flat'] = train['title'].apply(lambda text: cnouns.remove_headlines(text))
title = [tokenize(each[1]['title_flat']) for each in train.iterrows()]

## Training
1. Feature extraction - TfidVectorizer
2. Decomposition - PCA
3. Cluster - KMeans

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [ ]:
vectorizer = TfidfVectorizer(lowercase=False, ngram_range=(1,2))
title_flat = [item for sublist in title for item in sublist]
x_list = vectorizer.fit_transform(title_flat)

In [ ]:
x_list_100d = PCA(n_components=100).fit_transform(x_list.toarray())
x_list_100d.shape

### Scoring

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
from IPython.display import display, HTML
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

### Best Silhoutte Score

In [ ]:
best_score = 0.0
best_k = 0

In [ ]:
for k in range(15, 35):
    km = KMeans(n_clusters=k, n_jobs=-1).fit(x_list_100d)
    score = silhouette_score(x_list_100d, km.labels_)
    if best_score < score:
        best_score = score
        best_k = k
    print("In Clusters =", k, ", Score is : %0.3f" % score)
print("In Clusters =", best_k, ", Best score is : %0.3f" % best_score)

### K-Means Algorithm

In [ ]:
km = KMeans(n_clusters=best_k, n_jobs=-1).fit(x_list_100d)
labels = km.labels_
centroids = km.cluster_centers_
print(km.inertia_)

In [ ]:
x_list_vector = x_list_100d.tolist()
train = train.drop(['title_flat', 'target_str'], axis=1)
train['cluster'] = labels

## Choose Best Cluster
1. Cluster size < 500
2. Recent published
3. Minimum inertia

### Compare best cluster

In [ ]:
sample_silhouette_values = silhouette_samples(x_list_100d, labels)
sample_silhouette_score = []
best_cluster = []
cluster_num = best_k

for i in range(cluster_num):
    ith_cluster_silhouette_values = \
        sample_silhouette_values[labels == i]
        
    print('Cluster %d: ' % (i), abs(ith_cluster_silhouette_values.mean()))
    sample_silhouette_score.append(abs(ith_cluster_silhouette_values.mean()))

sample_silhouette_score.sort(reverse=True)
sample_silhouette_score = sample_silhouette_score[:8]

In [ ]:
sample_silhouette_score

In [ ]:
for i in range(cluster_num):
    ith_cluster_silhouette_values = \
        sample_silhouette_values[labels == i]
        
    if abs(ith_cluster_silhouette_values.mean()) in sample_silhouette_score:
        best_cluster.append(i)

In [ ]:
best_cluster

In [ ]:
train = train[train['cluster'].isin(best_cluster)]
train.cluster.unique()

## Result

In [ ]:
cluster_data = []

for cluster_index in range(cluster_num):
    if cluster_index in best_cluster:
        cluster_data.append(train[train['cluster'] == cluster_index])
    
for i, d in enumerate(cluster_data):
    print('Cluster %d:' % (i), 'Size %d' % (len(d)))

    display(d[['title', 'category']].sample(min(10, len(d))))
    print('\n\n')

## Save Dataframe to MongoDB

In [ ]:
from pymongo import MongoClient

In [ ]:
client = MongoClient('mongodb://localhost:27017/somanews')
client.somanews.authenticate('ssomanews', 'ssomanews1029')
db = client.get_database('somanews')
articles = db.get_collection('articles')

In [ ]:
articles.insert_many(train.to_dict(orient='records'))
client.close()